In [31]:
import cv2
import os
import json

def capture_significant_frames(video_path, output_folder, threshold_ratio=0.05, change_area_ratio=0.05, min_interval_sec=2):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Initialize the video capture
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file {video_path}")
        return

    # Get video frame rate
    fps = cap.get(cv2.CAP_PROP_FPS)
    min_frames_interval = int(fps * min_interval_sec)

    # Read the first frame and initialize variables
    ret, prev_frame = cap.read()
    if not ret:
        print("Cannot read video file")
        return
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_index = 0
    snapshot_index = 0
    last_snapshot_frame = -min_frames_interval

    keyframes_info = {}

    # Calculate the resolution-independent threshold and change area
    total_pixels = prev_gray.shape[0] * prev_gray.shape[1]
    threshold = 255 * threshold_ratio
    min_change_area = total_pixels * change_area_ratio

    while True:
        # Read next frame
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_diff = cv2.absdiff(prev_gray, gray)
        _, thresh_diff = cv2.threshold(frame_diff, threshold, 255, cv2.THRESH_BINARY)
        change_area = cv2.countNonZero(thresh_diff)

        if change_area > min_change_area and frame_index - last_snapshot_frame >= min_frames_interval:
            snapshot_filename = f"snapshot_{snapshot_index}.png"
            full_snapshot_path = os.path.join(output_folder, snapshot_filename)
            cv2.imwrite(full_snapshot_path, frame)
            print(f"Snapshot taken at frame {frame_index}: {full_snapshot_path}")

            # Convert frame_index to timestamp in hours:minutes:seconds
            total_seconds = int(frame_index / fps)
            hours = total_seconds // 3600
            minutes = (total_seconds % 3600) // 60
            seconds = total_seconds % 60
            timestamp = f"{hours:02d}:{minutes:02d}:{seconds:02d}"

            keyframes_info[snapshot_filename] = timestamp

            snapshot_index += 1
            last_snapshot_frame = frame_index

        prev_frame = frame.copy()
        prev_gray = gray.copy()
        frame_index += 1

    # Save keyframes info to JSON file
    json_filename = os.path.join(output_folder, "keyframes_info.json")
    try:
        with open(json_filename, 'w') as json_file:
            json.dump(keyframes_info, json_file, indent=4)
        print(f"Keyframe info saved to {json_filename}")
    except Exception as e:
        print(f"Error saving JSON file: {e}")

    cap.release()
    print("Processing complete.")

# Example usage:
# capture_significant_frames("path/to/video.mp4", "output/images", threshold_ratio=0.05, change_area_ratio=0.05)


In [33]:
capture_significant_frames(r"E:\Temp\Nvidia.mp4",r"E:\Temp\nvidia", threshold_ratio=0.25, change_area_ratio=0.03, min_interval_sec=5)

Snapshot taken at frame 331: E:\Temp\keyframes\snapshot_0.png
Snapshot taken at frame 2889: E:\Temp\keyframes\snapshot_1.png
Snapshot taken at frame 3785: E:\Temp\keyframes\snapshot_2.png
Snapshot taken at frame 4162: E:\Temp\keyframes\snapshot_3.png
Snapshot taken at frame 6199: E:\Temp\keyframes\snapshot_4.png
Snapshot taken at frame 6417: E:\Temp\keyframes\snapshot_5.png
Snapshot taken at frame 8765: E:\Temp\keyframes\snapshot_6.png
Snapshot taken at frame 9173: E:\Temp\keyframes\snapshot_7.png
Snapshot taken at frame 10935: E:\Temp\keyframes\snapshot_8.png
Snapshot taken at frame 11114: E:\Temp\keyframes\snapshot_9.png
Snapshot taken at frame 12237: E:\Temp\keyframes\snapshot_10.png
Snapshot taken at frame 12556: E:\Temp\keyframes\snapshot_11.png
Snapshot taken at frame 13090: E:\Temp\keyframes\snapshot_12.png
Snapshot taken at frame 14768: E:\Temp\keyframes\snapshot_13.png
Snapshot taken at frame 18244: E:\Temp\keyframes\snapshot_14.png
Snapshot taken at frame 18932: E:\Temp\keyfr